# 02. Upload polygon mask to cloud buckets

This notebook uploads the polygon mask created for each Sentinel-2 and Landsat tile into a GCP (Google Cloud Platform) cloud bucket.

This notebook reads in the output of the notebook [01_Create_polygon_mask.ipynb](https://github.com/ShiruiH/time-series-OFS/blob/main/01_Create_polygon_mask.ipynb), which is located in `/outputs/Sentinel2_tiles_mask` and `/outputs/Landsat_tiles_mask`.

In [27]:
# import libraries
import os
import glob
import datetime
from google.cloud import storage

Authenticate Google Cloud project

In [2]:
! gcloud auth login
! gcloud config set project nsw-dpe-gee-tst

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=eJJQiV1Bk14GOlVhiM0uVdDd66YoOv&access_type=offline&code_challenge=aXxW7W7dUjv9eGpv70UEduDjYDZxDM878h32LptYADw&code_challenge_method=S256


You are now logged in as [kilian.vos@dpie.nsw.gov.au].
Your current project is [nsw-dpe-gee-tst].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].


Set bucket and folder names

In [28]:
# Set GCS bucket name
BUCKET_NAME = f'label-tiles'

# Set satellite folder name, choose from: Sentinel2_tiles_mask; Landsat_tiles_mask
SATELLITE_FOLDER = 'Landsat_tiles_mask'

print(f'Name of the cloud bucket: {BUCKET_NAME}\nName of the folder: {SATELLITE_FOLDER}')

sat_name = SATELLITE_FOLDER.split('_')[0]

Name of the cloud bucket: label-tiles
Name of the folder: Landsat_tiles_mask


Create the folder in the bucket to store images

In [33]:
# Run this cell once to create the folder in the bucket
bucket = storage.Client().get_bucket(BUCKET_NAME)
blob = bucket.blob(f'{SATELLITE_FOLDER}/')
blob.upload_from_string('')
print(f"Folder '{SATELLITE_FOLDER}' is created in {BUCKET_NAME}")

Folder 'Landsat_tiles_mask' is created in label-tiles


Auxiliary function

In [34]:
# function to upload a local file to a cloud bucket
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client('')
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    generation_match_precondition = 0
    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)
    print(f'File {source_file_name} uploaded to {destination_blob_name}.')

Locate the polygon mask in `/outputs` folder

In [35]:
fp_outputs = os.path.join(os.getcwd(), 'outputs', 'polygon_masks')
print(fp_outputs)
# load Sentinel-2 tiles
fp_tiles = os.path.join(fp_outputs, SATELLITE_FOLDER)
fn_tiles = glob.glob(f'{fp_tiles}/*.tif')
print(f'{len(fn_tiles)} {sat_name} tiles found: {fn_tiles}')

c:\Shirui_Hao\GitHub\time-series-OFS\outputs\polygon_masks
3 Landsat tiles found: ['c:\\Shirui_Hao\\GitHub\\time-series-OFS\\outputs\\polygon_masks\\Landsat_tiles_mask\\LC08_091080_20231115.tif', 'c:\\Shirui_Hao\\GitHub\\time-series-OFS\\outputs\\polygon_masks\\Landsat_tiles_mask\\LC09_092081_20231115.tif', 'c:\\Shirui_Hao\\GitHub\\time-series-OFS\\outputs\\polygon_masks\\Landsat_tiles_mask\\LC09_092083_20231115.tif']


Load to the cloud bucket

In [36]:
for fn in fn_tiles:
    base_name = os.path.basename(fn)
    destination_blob = f'{SATELLITE_FOLDER}/{base_name}'
    upload_blob(BUCKET_NAME, fn, destination_blob)

File c:\Shirui_Hao\GitHub\time-series-OFS\outputs\polygon_masks\Landsat_tiles_mask\LC08_091080_20231115.tif uploaded to Landsat_tiles_mask/LC08_091080_20231115.tif.
File c:\Shirui_Hao\GitHub\time-series-OFS\outputs\polygon_masks\Landsat_tiles_mask\LC09_092081_20231115.tif uploaded to Landsat_tiles_mask/LC09_092081_20231115.tif.
File c:\Shirui_Hao\GitHub\time-series-OFS\outputs\polygon_masks\Landsat_tiles_mask\LC09_092083_20231115.tif uploaded to Landsat_tiles_mask/LC09_092083_20231115.tif.


### ***(OPTIONAL) Only run this cell if you need to delete all files in the bucket folder***


In [26]:
# Delete all objects in the folder if needed
bucket = storage.Client().get_bucket(BUCKET_NAME)
# List all objects in the folder
blobs = bucket.list_blobs(prefix=SATELLITE_FOLDER)

# Delete all objects in the folder
for blob in blobs:
    blob.delete()

print(f"All objects in folder '{SATELLITE_FOLDER}' have been deleted. Now you can re-create the folder and re-upload files in the bucket.")

All objects in folder 'Landsat_tiles_mask' have been deleted. Now you can re-upload files.
